In [ ]:
"""
Вы можете заметить, что ячейки здесь не выполнены, возникает ощущение, что я не прогонял этот код
Но это, конечно же, неправда
Так получилось, потому что изначально было два разных модуля: первый создавал директории, второй "раскладывал" 
примеры по этим директориям
Потом я подумал, что это выглядит не логично, и объединил их в единый модуль
Так как все папки уже были созданы, то код здесь не выполнен. 
"""

In [ ]:
import os
from pathlib import Path
import numpy as np
import glob
import shutil
from PIL import Image


import torch
from torch.utils.data import DataLoader
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as T
from torchvision.datasets import ImageFolder

In [ ]:
# какие классы?
classes = ["face", "no_face"]
# какие наборы?
datasets = ["train", "test"]

In [ ]:
DATA_DIR=Path("images")

def make_dirs(classes, datasets):
    for ds in datasets:
        for cls in classes:
            (DATA_DIR / ds / cls).mkdir(parents=True, exist_ok=True)

In [ ]:
make_dirs(classes, datasets)

In [ ]:
# перенос изображений и их переимнование
def move_just_images():
    os.mkdir("all_images")
    img_cntr = 0
    for png in glob.iglob(os.path.join("ThermalFaceDatabase", "*.png")):
        img_cntr +=1 
        shutil.copy(png, "all_images")
        
    return img_cntr

In [ ]:
# перенести все png-изображения в одну папку
n_images = move_just_images()

In [ ]:
# создание папки для pts-файлов
def make_pts():
    if os.path.exists('pts'):
        return "path is already exists"
    os.makedirs("pts")

    for (idx, pts) in enumerate(sorted(glob.iglob(os.path.join("ThermalFaceDatabase", "*.pts")))):
        shutil.copy(pts, f"pts/{idx}.jpg_lfb.pts")



In [ ]:
make_pts()

In [1]:
# DEPRECATED

# создание 2935 файлов, в каждом лежит png и соотв ему pts

# для каждой двойки файлов (png, pts) создать свою директориую в 'files' с соотв индексом
# for (idx, file) in enumerate(sorted(os.listdir("ThermalFaceDatabase"))):
#     if idx % 3 == 0:
#         os.makedirs(f"files/{idx//3}")
#     else:
#         file_idx = idx // 3
#         shutil.copy(f"ThermalFaceDatabase/{file}", f"files/{file_idx}/{file_idx}.{file.split('.',1)[1]}")

In [ ]:
# ф-ия для парсинга pts файлов
# вернет первые координаты (два числа) рта
def take_coords_from_pts(file_path):
    with open(file_path) as pts:
        arr_of_coords = pts.readlines()[3:-1]
        coords = (float(arr_of_coords[0].split()[0]),
                  float(arr_of_coords[0].split()[1]))
        return coords

In [ ]:
# 4 разных ф-ий для обрезки фото для класса no_face
def crop_noface_1(img, points, win_size=(300, 300)):
    x = points[0]
    y = points[1]
    new_img = img.crop((x-win_size[0]/2, y+win_size[0]/2, x+win_size[1]/2, y+3/2*win_size[1]))
    
    return new_img

def crop_noface_2(img, points, win_size=(300, 300)):
    x = points[0]
    y = points[1]
    new_img = img.crop((x-3/2*win_size[0], y+win_size[0]/2, x-win_size[1]/2, y+3/2*win_size[1]))
    
    return new_img

def crop_noface_3(img, points, win_size=(300, 300)):
    x = points[0]
    y = points[1]
    new_img = img.crop((x-win_size[0]*2, y+win_size[0]/2, x-win_size[1], y+3/2*win_size[1]))
    
    return new_img

def crop_noface_4(img, points, win_size=(300, 300)):
#     x = points[0]
#     y = points[1]
#     new_img = img.crop((x+win_size[0]/2, y+win_size[0]/2, x+win_size[1], y+win_size[1]))
    x = points[0]
    y = points[1]
    new_img = img.crop((x-win_size[0], y, x-win_size[1]/2, y+win_size[1]))
    
    return new_img

In [ ]:
# перенести изобрадения в тренировочный класс "нелицо"
def move_to_train_noface(size):

    # размеры окон, из к-ых получатся изображения без лица
    available_win_sizes = [200, 300]
    # для каждого изображения
    for (idx, png) in enumerate(sorted(glob.glob(os.path.join("ThermalFaceDatabase", "*.png")))[:size]):
        # откроем его
        img = Image.open(png)
        # возьмем координаты рта из соотв файла pts
        mouth_points = take_coords_from_pts(f"pts/{idx}.jpg_lfb.pts")
        # для каждого размера окна
        for win_size in available_win_sizes:
            # обрезаем его 1
            cropped_img1 = crop_noface_1(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img1.save(f"images/train/no_face/{idx}_{win_size}_0.png")
            
            # обрезаем его 2
            cropped_img2 = crop_noface_2(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img2.save(f"images/train/no_face/{idx}_{win_size}_1.png")
            
            # обрезаем его 3
            cropped_img3 = crop_noface_3(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img3.save(f"images/train/no_face/{idx}_{win_size}_2.png")
            
            # обрезаем его 4
            cropped_img4 = crop_noface_4(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img4.save(f"images/train/no_face/{idx}_{win_size}_3.png")

In [ ]:
# перенести в тестовый класс "нелицо"
def move_to_test_noface(size):
    # размеры окон, из к-ых получатся изображения с лицом
    available_win_sizes = [200, 300]
    # для каждого изображения
    for (idx, png) in enumerate(sorted(glob.glob(os.path.join("ThermalFaceDatabase", "*.png")))[size:]):

        # откроем его
        img = Image.open(png)
        # возьмем координаты рта из соотв файла pts
        mouth_points = take_coords_from_pts(f"pts/{idx+size}.jpg_lfb.pts")
        # для каждого размера окна
        for win_size in available_win_sizes:
            # обрезаем его 1
            cropped_img1 = crop_noface_1(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img1.save(f"images/test/no_face/{idx}_{win_size}_0.png")
            
            # обрезаем его 2
            cropped_img2 = crop_noface_2(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img2.save(f"images/test/no_face/{idx}_{win_size}_1.png")
            
            # обрезаем его 3
            cropped_img3 = crop_noface_3(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img3.save(f"images/test/no_face/{idx}_{win_size}_2.png")
            
            # обрезаем его 4
            cropped_img4 = crop_noface_4(img, mouth_points, win_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img4.save(f"images/test/no_face/{idx}_{win_size}_3.png")

In [ ]:
# перенести в тестовый класс "лицо"
def move_to_test_face(size):
    # размеры окон, из к-ых получатся изображения с лицом
    available_win_sizes = [100, 200, 300, 500]
    # для каждого изображения
    for (idx, png) in enumerate(sorted(glob.glob(os.path.join("ThermalFaceDatabase", "*.png")))[size:]):
        png = Image.open(png)
        # возьмем координаты рта из соотв файла pts
        mouth_points = take_coords_from_pts(f"pts/{idx+size}.jpg_lfb.pts")
        # для каждого размера окна
        for win_size in available_win_sizes:
            # обрезаем его на уровне рта, получая новое изображение
            cropped_img1 = crop_image1(png, mouth_points, window_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img1.save(f"images/test/face/{idx}_{win_size}_0.png")
            # обрезаем его выше рта, получая новое изображение
            cropped_img_up = crop_image_up(png, mouth_points, window_size=(win_size, win_size))
            # сохраням в соотв директории
            cropped_img_up.save(f"images/test/face/{idx}_{win_size}_1.png")

In [ ]:
# обрезка фото с центром на уровне рта, 
# points - координаты рта
# window_size - размер окна
# вернет новое обрезанное фото
def crop_image1(img, points, window_size=(300, 300)):
    x = points[0]
    y = points[1]
    new_img = img.crop((x-window_size[0]/2, y-window_size[0]/2, 
                        x+window_size[1]/2, y+window_size[1]/2))

    return new_img

# обрезка фото с центром выше рта, 
# mouth_points - координаты рта
# window_size - размер окна
# вернет новое обрезанное фото
def crop_image_up(img, mouth_points, window_size=(300, 300)):
    x = mouth_points[0]
    y = mouth_points[1]
    new_img = img.crop((x-window_size[0]/2, y-window_size[0], 
                        x+window_size[1]/2, y))

    return new_img

In [ ]:
# Заняло: 2 минуты
%%time
TRAIN_SIZE = int(0.8 * 2935)

move_to_train_noface(TRAIN_SIZE)

In [ ]:
# Заняло: 42 сек
%%time

move_to_test_noface(TRAIN_SIZE)

In [ ]:
# Заняло: 2 минуты
%%time
# обрезать и сохранить изображения в images/test/face
move_to_test_face(TRAIN_SIZE)